In [1]:
import boto3
import pendulum as pt
import numpy as np
import pandas as pd
from ds_utils.db.connectors import PivotDW
import datetime as dt






In [3]:
premium_share_df = pd.read_csv('/Users/trevor/data/Assumptions/retained_by_pivot.csv')

premium_share_df = premium_share_df[['Carrier', 'State', '2019-01']]
premium_share_df.rename(columns={'Carrier': 'carrier_code', 'State': 'state_code', '2019-01': 'fee_share'}, inplace=True)

premium_share_df['effective_date'] = dt.datetime.strptime('2019-01', '%Y-%m')
premium_share_df['expires_date'] = np.nan
premium_share_df = premium_share_df[['carrier_code', 'state_code', 'effective_date', 'expires_date', 'fee_share']]

premium_share_df.head()

,carrier_code,state_code,effective_date,expires_date,fee_share
0,CLIC,AK,2019-01-01,NaN,0.3
1,CLIC,AL,2019-01-01,NaN,0.3
2,CLIC,AR,2019-01-01,NaN,0.3
3,CLIC,AZ,2019-01-01,NaN,0.3
4,CLIC,CA,2019-01-01,NaN,0.3


In [4]:
recur_fee_df = pd.read_csv('/Users/trevor/data/Assumptions/recurring_fee_retained_by_pivot.csv')
recur_fee_df.rename(
    columns={
        'Carrier': 'carrier_code',
        'Policy sold through Adroit': 'is_androit',
        'State': 'state_code'
        },
    inplace=True
)

dt_cols = list(recur_fee_df.columns)[3:]

dfs = list()
for col in dt_cols:
    df_slice = recur_fee_df[['carrier_code', 'is_androit', 'state_code', col]].copy()
    df_slice['effective_dt'] = dt.datetime.strptime(col, '%Y-%m')
    df_slice.rename(columns={col: 'fee_share'}, inplace=True)
    dfs.append(df_slice[['carrier_code', 'is_androit', 'state_code', 'effective_dt', 'fee_share']])
    
recur_fee_df = pd.concat(dfs)

recur_fee_df = recur_fee_df.sort_values(
        by=['carrier_code', 'is_androit', 'state_code', 'effective_dt'],
        ascending=True
    )

recur_fee_df['previous_fee'] = recur_fee_df.groupby(['carrier_code', 'is_androit', 'state_code'])['fee_share'].shift(1)


recur_fee_df.set_index(['carrier_code', 'is_androit', 'state_code'], inplace=True)
recur_fee_df = recur_fee_df[recur_fee_df['fee_share'] != recur_fee_df['previous_fee']]
recur_fee_df.reset_index(inplace=True)
recur_fee_df['expiration_dt'] = recur_fee_df.groupby(['carrier_code', 'is_androit', 'state_code'])['effective_dt'].shift(-1)
recur_fee_df['expiration_dt'] = recur_fee_df['expiration_dt'] - pd.to_timedelta(1, unit='d')

recur_fee_df = recur_fee_df[['carrier_code', 'is_androit', 'state_code', 'effective_dt', 'expiration_dt', 'fee_share']]

recur_fee_df.set_index(['carrier_code', 'is_androit', 'state_code'], inplace=True)
recur_fee_df.head(100)

effective_dt expiration_dt  fee_share
carrier_code is_androit state_code                                      
CLIC         False      AK           2019-01-01    2019-02-28      11.99
                        AK           2019-03-01           NaT       4.95
                        AL           2019-01-01    2019-02-28      11.99
                        AL           2019-03-01           NaT       4.95
                        AR           2019-01-01    2019-02-28      11.99
                        AR           2019-03-01           NaT       4.95
                        AZ           2019-01-01    2019-02-28      11.99
                        AZ           2019-03-01           NaT       4.95
                        CA           2019-01-01    2019-02-28      11.99
                        CA           2019-03-01           NaT       4.95
                        CO           2019-01-01    2019-02-28      11.99
                        CO           2019-03-01           NaT       4.95
                        CT           2019-01-01    2019-02-28      11.99
                        CT           2019-03-01           NaT       4.95
                        DC           2019-01-01    2019-02-28      11.99
                        DC           2019-03-01           NaT       4.95
                        DE           2019-01-01    2019-02-28      11.99
                        DE           2019-03-01           NaT       4.95
                        FL           2019-01-01    2019-02-28      11.99
                        FL           2019-03-01           NaT       4.95
                        GA           2019-01-01    2019-02-28      11.99
                        GA           2019-03-01           NaT       4.95
                        HI           2019-01-01    2019-02-28      11.99
                        HI           2019-03-01           NaT       4.95
                        IA           2019-01-01    2019-02-28      11.99
                        IA           2019-03-01           NaT       4.95
                        ID           2019-01-01    2019-02-28      11.99
                        ID           2019-03-01           NaT       6.95
                        IL           2019-01-01    2019-02-28      11.99
                        IL           2019-03-01           NaT       4.95
                        IN           2019-01-01    2019-02-28      11.99
                        IN           2019-03-01           NaT       4.95
                        KS           2019-01-01    2019-02-28      11.99
                        KS           2019-03-01           NaT       4.95
                        KY           2019-01-01    2019-02-28      11.99
                        KY           2019-03-01           NaT       4.95
                        LA           2019-01-01    2019-02-28      11.99
                        LA           2019-03-01           NaT       4.95
                        MA           2019-01-01    2019-02-28      11.99
                        MA           2019-03-01           NaT       4.95
                        MD           2019-01-01    2019-02-28      11.99
                        MD           2019-03-01           NaT       4.95
                        ME           2019-01-01    2019-02-28      11.99
                        ME           2019-03-01           NaT       4.95
                        MI           2019-01-01    2019-02-28      11.99
                        MI           2019-03-01           NaT       4.95
                        MN           2019-01-01    2019-02-28      11.99
                        MN           2019-03-01           NaT       4.95
                        MO           2019-01-01    2019-02-28      11.99
                        MO           2019-03-01           NaT       6.95
                        MS           2019-01-01    2019-02-28      11.99
                        MS           2019-03-01           NaT       4.95
                        MT           2019-01-01    2019-02-28      11.99
                    

In [5]:
enroll_fee_df = pd.read_csv('/Users/trevor/data/Assumptions/enrollment_fee_retained_by_pivot.csv')
enroll_fee_df.rename(
    columns={
        'Carrier': 'carrier_code',
        'Policy sold through Adroit': 'is_androit',
        'State': 'state_code'
        },
    inplace=True
)

dt_cols = list(enroll_fee_df.columns)[3:]

dfs = list()
for col in dt_cols:
    df_slice = enroll_fee_df[['carrier_code', 'is_androit', 'state_code', col]].copy()
    df_slice['effective_dt'] = dt.datetime.strptime(col, '%Y-%m')
    df_slice.rename(columns={col: 'fee_share'}, inplace=True)
    dfs.append(df_slice[['carrier_code', 'is_androit', 'state_code', 'effective_dt', 'fee_share']])
    
enroll_fee_df = pd.concat(dfs)

enroll_fee_df = enroll_fee_df.sort_values(
        by=['carrier_code', 'is_androit', 'state_code', 'effective_dt'],
        ascending=True
    )

enroll_fee_df['previous_fee'] = enroll_fee_df.groupby(['carrier_code', 'is_androit', 'state_code'])['fee_share'].shift(1)


enroll_fee_df.set_index(['carrier_code', 'is_androit', 'state_code'], inplace=True)
enroll_fee_df = enroll_fee_df[enroll_fee_df['fee_share'] != enroll_fee_df['previous_fee']]
enroll_fee_df.reset_index(inplace=True)
enroll_fee_df['expiration_dt'] = enroll_fee_df.groupby(['carrier_code', 'is_androit', 'state_code'])['effective_dt'].shift(-1)
enroll_fee_df['expiration_dt'] = enroll_fee_df['expiration_dt'] - pd.to_timedelta(1, unit='d')

enroll_fee_df = enroll_fee_df[['carrier_code', 'is_androit', 'state_code', 'effective_dt', 'expiration_dt', 'fee_share']]

enroll_fee_df.set_index(['carrier_code', 'is_androit', 'state_code'], inplace=True)
enroll_fee_df.head(100)

effective_dt expiration_dt  fee_share
carrier_code is_androit state_code                                      
CLIC         False      AK           2019-01-01    2019-02-28      15.00
                        AK           2019-03-01           NaT      19.95
                        AL           2019-01-01    2019-02-28      15.00
                        AL           2019-03-01           NaT      19.95
                        AR           2019-01-01    2019-02-28      15.00
                        AR           2019-03-01           NaT      19.95
                        AZ           2019-01-01    2019-02-28      15.00
                        AZ           2019-03-01           NaT      19.95
                        CA           2019-01-01    2019-02-28      15.00
                        CA           2019-03-01           NaT      19.95
                        CO           2019-01-01    2019-02-28      15.00
                        CO           2019-03-01           NaT      19.95
                        CT           2019-01-01    2019-02-28      15.00
                        CT           2019-03-01           NaT      19.95
                        DC           2019-01-01    2019-02-28      15.00
                        DC           2019-03-01           NaT      19.95
                        DE           2019-01-01    2019-02-28      15.00
                        DE           2019-03-01           NaT      19.95
                        FL           2019-01-01    2019-02-28      15.00
                        FL           2019-03-01           NaT      19.95
                        GA           2019-01-01    2019-02-28      15.00
                        GA           2019-03-01           NaT      19.95
                        HI           2019-01-01    2019-02-28      15.00
                        HI           2019-03-01           NaT      19.95
                        IA           2019-01-01    2019-02-28      15.00
                        IA           2019-03-01           NaT      19.95
                        ID           2019-01-01    2019-02-28      15.00
                        ID           2019-03-01           NaT      19.95
                        IL           2019-01-01    2019-02-28      15.00
                        IL           2019-03-01           NaT      19.95
                        IN           2019-01-01    2019-02-28      15.00
                        IN           2019-03-01           NaT      19.95
                        KS           2019-01-01    2019-02-28      15.00
                        KS           2019-03-01           NaT      19.95
                        KY           2019-01-01    2019-02-28      15.00
                        KY           2019-03-01           NaT      19.95
                        LA           2019-01-01    2019-02-28      15.00
                        LA           2019-03-01           NaT      19.95
                        MA           2019-01-01    2019-02-28      15.00
                        MA           2019-03-01           NaT      19.95
                        MD           2019-01-01    2019-02-28      15.00
                        MD           2019-03-01           NaT      19.95
                        ME           2019-01-01    2019-02-28      15.00
                        ME           2019-03-01           NaT      19.95
                        MI           2019-01-01    2019-02-28      15.00
                        MI           2019-03-01           NaT      19.95
                        MN           2019-01-01    2019-02-28      15.00
                        MN           2019-03-01           NaT      19.95
                        MO           2019-01-01    2019-02-28      15.00
                        MO           2019-03-01           NaT      19.95
                        MS           2019-01-01    2019-02-28      15.00
                        MS           2019-03-01           NaT      19.95
                        MT           2019-01-01    2019-02-28      15.00
                    

In [ ]:
# underwriting profit
['product', 'effective_dt', 'expiration_dt', 'fee_share']
['STM', '2019-01-01', np.nan, 0.0466]




In [ ]:
ltv_params_create = """
 CREATE TABLE ltv_parameters(
     param_type VARCHAR(20),
     state_code VARCHAR(2),
     carrier VARCHAR(30),
     is_androit BOOLEAN,
     policy_type VARCHAR(20),
     effective_date DATE,
     end_date DATE,
     flat_amount NUMERIC(8,4),
     percent_amount NUMERIC(5,4),
     is_recurring BOOLEAN
 )

"""


In [ ]:
from ds_utils.db.connectors import PivotDW

with PivotDW() as database_context:
    
    database_context.load_df(df, table, schema)
    
    
    
    


In [ ]:
list(response)